In [6]:
import itertools
import numpy as np
import pandas as pd
import scipy.stats as stats
import matplotlib.pyplot as plt
import math

# PROBLEM:

<br/>

<b> Probability based motif finding is not accurate as it does not take into consideration other factors leading false positive motfs.
    
Incorporating binding energies makes TFBS finding more accurate.
<br/>
<br/>

<u>REFERENCES</u>:</b>

An efficient algorithm for improving structure-based prediction of transcription factor binding sites[KMERSUM]:-

https://bmcbioinformatics.biomedcentral.com/articles/10.1186/s12859-017-1755-0/figures/3


TFBS Models:
Reference - https://doi.org/10.1016/j.molp.2018.10.010

<br/>
<br/>
<center>KMERSUM ALGO</center>
<img src="ALGO.png">

# DATA

Getting energy table for interaction between all AA and Nucleotide bases by using AMINO ACID–NUCLEOTIDE INTERACTION ANALYSIS TOOL:-

https://academic.oup.com/nar/article/45/W1/W388/3787858#90594982

To test the algo we select the protein motif:

cAMP responsive element binding protein 1[CREB1] bZIP famility motif : VTKACGTMA 

http://humantfs.ccbr.utoronto.ca/myTFPageUniversal.php?eID=ENSG00000118260
http://humantfs.ccbr.utoronto.ca/myTFPageUniversal.php?eID=ENSG00000118260

In [7]:
"""Table made for the interaction energies of AA and Nucleotide bases by using AMINO ACID–NUCLEOTIDE INTERACTION ANALYSIS TOOL """
pd.read_csv("ACB_energy_aa_nucleotide1.csv")

,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
0,3.6,7.2,24.4,15.1,8.6,3.5,7.5,2.9,23.9,5.4,3.7,2.8,7.0,-0.2,20.2,1.3,6.7,5.0,9.2,2.6
1,3.4,7.4,80.5,92.5,4.3,2.9,5.6,2.9,-13.8,6.2,8.3,4.3,7.0,2.0,-11.8,0.7,3.4,5.6,7.8,4.0
2,5.8,4.5,65.2,-1.4,13.6,4.3,3.2,5.1,133.3,4.0,7.2,-5.1,7.9,-0.3,129.7,-0.3,7.1,4.4,9.6,6.8
3,3.3,4.5,9.5,14.6,8.7,2.6,6.5,5.0,42.1,4.9,7.6,0.8,7.6,7.4,29.5,0.3,4.7,3.3,9.5,9.5


# complexity

It is a Big(O) of n^3

In [8]:
# genrating a list 5mers that are requeired in kmersum

permutations = ["".join(list(_))for _ in itertools.product('ACGT', repeat = 9)]

all_fivemers=[]
for ninemer in permutations:
    fivemers = [ninemer[i:i+5] for i in range(5)]
    all_fivemers.append(fivemers)

In [9]:
#making energy table for the provided protein motif
motif_seq = 'VTKACGTMA '
complete_energy_table=pd.read_csv("ACB_energy_aa_nucleotide1.csv")
energies = complete_energy_table.loc[:, [motif_seq[0],motif_seq[2],motif_seq[2],motif_seq[3],motif_seq[4],motif_seq[5],motif_seq[6],motif_seq[7],motif_seq[8]]]
energies.to_csv("current_9mer_energy.csv",sep=",",header=None,index=False)
energies=pd.read_csv("current_9mer_energy.csv",header=None)
energies

,0,1,2,3,4,5,6,7,8
0,5.0,23.9,23.9,3.6,7.2,3.5,6.7,3.7,3.6
1,5.6,-13.8,-13.8,3.4,7.4,2.9,3.4,8.3,3.4
2,4.4,133.3,133.3,5.8,4.5,4.3,7.1,7.2,5.8
3,3.3,42.1,42.1,3.3,4.5,2.6,4.7,7.6,3.3


# Creating a distributed energy table for the 9mers

In [10]:
#kmersum algo

distributed_e = []
for fivemers,ninemer in zip(all_fivemers,permutations):
    total_e = 0
    for idx_1, fivemer in enumerate(fivemers):
        fivemer_e = 0
        for idx_2, base in enumerate(fivemer):
            if base == 'A':
                fivemer_e += energies[idx_1+idx_2][0]
            elif base == 'C':
                fivemer_e += energies[idx_1+idx_2][1]
            elif base == 'G':
                fivemer_e += energies[idx_1+idx_2][2]
            else :
                fivemer_e += energies[idx_1+idx_2][3]
        total_e += fivemer_e
    distributed_e.append((ninemer,total_e))

df = pd.DataFrame(
    {
    '9mer' : [x[0] for x in distributed_e],
    'energy' : [x[1] for x in distributed_e]
})

# removing the seq which have lower energy than the 99%tile and removing the first 2 and the last 2 bases from the seq

In [11]:
p = np.percentile(np.array(df['energy']),99)
df_quardt_4 = df[df['energy'] >= p]
sequences = [seq[2:-2] for seq in df_quardt_4['9mer']]

In [12]:
#cutoof energy for the 5mers
p

774.5000000000002

In [13]:
#making frequency matrix and probability matrix

def con_table_updater(string,table):
    for index,base in enumerate(string):
        if base == "A":
            table[0][index] += 1
        elif base == "C":
            table[1][index] += 1
        elif base == "G":
            table[2][index] += 1
        else:
            table[3][index] += 1   
            
    
cTable = [[0]*5 for _ in range(4)]
    
for seq in sequences:
    con_table_updater(seq,cTable)
    
pfm = np.matrix(cTable)
print(f"Frequency Matrix: \n{pfm}")
    
ppm = np.divide(np.matrix(cTable),len(sequences))
print(f"probablity Matrix: \n{ppm}")

pd.DataFrame(ppm).to_csv("matrix.txt",sep="\t",header=None,index=False)

Frequency Matrix: 
[[   0  442 1176  770  969]
 [   0  369 1368  423  140]
 [2622 1480   39 1125 1163]
 [   0  331   39  304  350]]
probablity Matrix: 
[[0.         0.16857361 0.44851259 0.29366895 0.36956522]
 [0.         0.14073227 0.52173913 0.16132723 0.05339436]
 [1.         0.56445461 0.01487414 0.42906178 0.44355454]
 [0.         0.12623951 0.01487414 0.11594203 0.13348589]]


# Producing Motif Logo using R package(seqLogo)

#Using seqLogo to produce motif logo

<img src="MOTIF LOGO.png">

# Conclusion


<b> The Motif Logo found is similar to GC Box which is seen to regulate the binding of CREB1:</b>
<br/><br/>
Reference - http://www.jbc.org/content/276/9/6350.short

We have used interactive energy to predict the binding sequence of the Trascription factor
We intended to use integrative energy to further increase the accuracy of TFBS.
Integrative energy takes into account the electrostatic energies and multibody potential of the DNA structure to predict the TFBS.

# Reproducibility
We need a 9 amino acid long chain that determines its TFBS using this algorithm, of all the motifs we ran on this algorithm we could only confirm CREB1 TFBS.